First, establish an ssh tunnel [with your PNI password] to jtb3-dev.princeton.edu to make the database available to you:

    ssh username@jtb3-dev.princeton.edu -L 3306:127.0.0.1:3306 -N

jtb3-dev is the database server on which all behavior and mouse data is stored. Second, make sure you have installed datajoint >= '0.12.dev3'
[ pip install datajoint==0.12.dev3 ]

In [18]:
import pylab as pl
import datajoint as dj
import numpy as np
from flask_login import UserMixin
assert dj.__version__=='0.12.dev3'

In [28]:
schema = dj.schema('ahoag_lightsheet_demo')

In [29]:
@schema
class User(dj.Lookup,UserMixin):
    definition = """
    # Users of the light sheet microscope
    username : varchar(20)      # user in the lab
    ---
    email       : varchar(50)
    """
    

@schema
class Experiment(dj.Manual):
    definition = """ # Experiments performed using the light sheet microscope
    -> User
    experiment : int    # allowed here are sql datatypes.
    ----
    species                 :   varchar(50)
    clearing_protocol       :   enum("iDISCO+","iDISCO for non-oxidizable fluorophores")
    fluorophores            :   varchar(100)
    primary_antibody        :   varchar(100)
    secondary_antibody      :   varchar(100)
    image_resolution        :   enum("1.3x","4x")
    """

In [30]:
Experiment() #Shows the current content of the database

username user in the lab,experiment allowed here are sql datatypes.,species,clearing_protocol,fluorophores,primary_antibody,secondary_antibody,image_resolution


## 2) Generate content for the database

In [31]:
home_dir = '/home/ahoag/Documents/Datajoint/'
csv_file = home_dir + '/pni_core_facility_request_responses.txt'
import csv

In [8]:
def fill_user_table():
    with open(csv_file, mode='r') as infile:
        reader = csv.reader(infile)
        data_dict = {}
        next(reader) # skips the header
        index = 0
        for row in reader:
            email = row[0]
            username = email.split('@')[0].lower()
            insert_user_list = [username,email]
#             print(insert_user_list)
            User().insert((insert_user_list,),skip_duplicates = True)
            index+=1

def fill_exp_table():
    with open(csv_file, mode='r') as infile:
        reader = csv.reader(infile)
        data_dict = {}
        next(reader) # skips the header
    #     rows_2insert = [row[0]]
        index = 0
        for row in reader:
            email = row[0]
            username = email.split('@')[0].lower()
            insert_exp_list = [username,index] + row[2:]  
#             print(insert_exp_list)
            Experiment().insert((insert_exp_list,))
            index+=1
    #     response_dict = {rows[0]:rows[1] for rows in reader}
    # print(keys)

In [9]:
fill_user_table()

In [13]:
User()

username user in the lab,email
ahoag,ahoag@princeton.edu
ejdennis,Ejdennis@princeton.edu
jverpeut,jverpeut@princeton.edu
willmore,Willmore@princeton.edu


In [11]:
fill_exp_table()

In [12]:
Experiment()

username user in the lab,experiment allowed here are sql datatypes.,species,clearing_protocol,fluorophores,primary_antibody,secondary_antibody,image_resolution
ejdennis,2,rat,iDISCO for non-oxidizable fluorophores,None,None,None,1.3x
jverpeut,0,mouse,iDISCO+,None,Anti cfos 1:1000,Donkey antirabbit AF647,1.3x
jverpeut,3,mouse,iDISCO+,GFP,Chicken antiGFP Aves 1:1000,Donkey anti chicken 647 1:750,1.3x
willmore,1,mouse,iDISCO for non-oxidizable fluorophores,GCampxDat-Cre Double Transgentic Mouse,None,None,1.3x
